In [42]:
# Patent grant data files are uploaded with Tuesday's date every week. This function gives the urls for all year
# Saved lot of time instead of manually getting all urls from USPTO website
from datetime import date, timedelta

def gettuesdays(year):
    d = date(year, 1, 1)                    # January 1st
    #print(d)
    #d += timedelta(days = 3 - d.weekday())  # First Thursday
    d += timedelta(days = (1 - d.weekday() if d.weekday() <= 1 else 7 + 1 - d.weekday()))
    wk = 1
    while d.year == year and (date.today() - d).days > 0:
        url = 'https://bulkdata.uspto.gov/data/patent/grant/redbook/bibliographic/'
        yield url + str(year) + '/ipgb' + str(d).replace('-','') + '_wk' + str(wk).zfill(2) + '.zip'
        d += timedelta(days = 7)
        wk += 1

def get_urls_for_year(year):
    urls = []
    for d in gettuesdays(year):
        urls.append(d)
    return urls
    #return urls[:2]

print(get_urls_for_year(2017)[0])
# for d in allthursdays(2016):
#         print(d)        
#Ref: #https://stackoverflow.com/questions/2003870/how-can-i-select-all-of-the-sundays-for-a-year-using-python

https://bulkdata.uspto.gov/data/patent/grant/redbook/bibliographic/2017/ipgb20170103_wk01.zip


In [43]:
!mkdir /home/ubuntu/data/patentgrantloadzone
!mkdir /home/ubuntu/data/output

mkdir: cannot create directory ‘/home/ubuntu/data/patentgrantloadzone’: File exists
mkdir: cannot create directory ‘/home/ubuntu/data/output’: File exists


In [44]:
# function to parse an xml file and write data to csv file
from datetime import datetime

def xmltocsv(inputfilename, filename):
    
    fName = '/home/ubuntu/data/output/patent_grant_data_' + filename[:-4] + '.csv'
    fw = open(fName, 'w')
    w = csv.writer(fw, delimiter = ',')

    dfilepath = inputfilename
    dfilepathtmp = dfilepath + 'tmp'
    !grep -v '<?xml version="1.0" encoding="UTF-8"?>' $dfilepath > $dfilepathtmp; mv $dfilepathtmp $dfilepath
    !sed -e '/^<!DOCTYPE/d' $dfilepath > $dfilepathtmp; mv $dfilepathtmp $dfilepath
    !sed -i '1 i\<items>' $dfilepath
    !sleep 5
    #!sed -i -e '$a</items>' $dfilepath
    !sed -i '$ a \</items>' $dfilepath
    
    patent_number = ''
    patent_doc_kind = ''
    patent_date = ''
    appl_doc_number = ''
    appl_date = ''
    length_of_grant = ''
    references_count = ''
    number_of_claims = ''
    number_of_days = ''
    count = 0
    for event, element in etree.iterparse(inputfilename, tag="us-patent-grant"):
        break_inner_for = False
        for child in element:
            
            if child.tag == 'us-bibliographic-data-grant':
                for child1 in child:
                    if child1.tag == 'publication-reference':
                        for child11 in child1:
                            if child11.tag == 'document-id':
                                for child111 in child11:
                                    if child111.tag == 'doc-number':
                                        patent_number = child111.text
                                    if child111.tag == 'kind':
                                        patent_doc_kind = child111.text    
                                    if child111.tag == 'date':
                                        patent_date = child111.text

                    if child1.tag == 'application-reference':
                        for child11 in child1:
                            for child11 in child1:
                                if child11.tag == 'document-id':
                                    for child111 in child11:
                                        if child111.tag == 'doc-number':
                                            appl_doc_number = child111.text
                                        if child111.tag == 'date':
                                            appl_date = child111.text
                    #print(child1.tag)
                    if child1.tag == 'us-term-of-grant':
                        for child11 in child1:
                            if child11.tag == 'length-of-grant':
                                length_of_grant = child11.text

                    
                    if child1.tag == 'us-references-cited':
                        references_count = len(child1)
                        #ref_text = etree.tostring(child).decode("utf-8")
                        #ref_count = ref_text.count('<us-citation>')
                        
                    if child1.tag == 'number-of-claims': 
                        number_of_claims = child1.text
                        
                    
                    if child1.tag == 'number-of-claims':
                        try:
                            pat_number_of_days = abs((datetime.strptime(patent_date, "%Y%m%d") - datetime.strptime(appl_date, "%Y%m%d")).days)
                        except ValueError:
                            number_of_days = 0
                        w.writerow([appl_doc_number, appl_date, patent_number, patent_date,patent_doc_kind, pat_number_of_days,length_of_grant,references_count,number_of_claims])
                        patent_number = ''
                        patent_doc_kind = ''
                        patent_date = ''
                        appl_doc_number = ''
                        appl_date = ''
                        length_of_grant = ''
                        references_count = ''
                        number_of_claims = ''
                        number_of_days = ''
                        break_inner_for = True
                        break
                if break_inner_for == True:
                    break
#             count += 1
#             if count > 3:
#                 break
        element.clear()
    fw.close() 

In [52]:
# function to load data for a year
import csv
import time
import os
import zipfile
import lxml.etree as etree, lxml.html
import string

def load_for_year(year_urls):
    indir = '/home/ubuntu/data/'
    loaddir = '/home/ubuntu/data/patentgrantloadzone'
    loaddelete = '/home/ubuntu/data/patentgrantloadzone/*'

    #for i in range(0,1):    
    for i in range(0,len(year_urls)):
        #print('Downloading file:' + str(i))
        #down load the file and 
        #dataurl = dataurlspath + dataurls[i]
        dataurl = year_urls[i]
        !curl -s -L $dataurl -o '/home/ubuntu/data/datafile.zip'

        for path, subdirs, files in os.walk(indir):

            for f in files:
                filepath = os.path.join(path, f)
                #print(filepath)
                !rm -r $loaddelete
                if filepath.endswith('.zip'):
                    zip_ref = zipfile.ZipFile(filepath, 'r')
                    zip_ref.extractall('/home/ubuntu/data/patentgrantloadzone')
                    zip_ref.close()

                datadir = '/home/ubuntu/data/patentgrantloadzone'
                for dpath, dsubdirs, dfiles in os.walk(datadir):
                    for df in dfiles:
                        if df.endswith('xml'):
                            dfilepath = os.path.join(dpath, df)
                            #print('Starting file: ',dfilepath )
                            #xmltocsv(dfilepath,dataurls[i])
                            url = year_urls[i]
                            flname = url[url.rfind("/")+1:]
                            xmltocsv(dfilepath,flname)
                            print('Loaded file: ', dfilepath )
            break  

In [53]:
# call the function for every year from 2017 back to 2001
# We only loaded data until 2005

for year in reversed(range(2005,2018)):    
    #delete output folder before starting a new loop
    !rm /home/ubuntu/data/output/*
    
    load_for_year(get_urls_for_year(year))
    
    #concatenate all week files into a year file
    year_filename = '/home/ubuntu/data/output/patent_grant_data_' + str(year) + '.csv'
    !cat /home/ubuntu/data/output/*.csv > $year_filename
    #!sed  -i '1i appl_doc_number, appl_date, patent_number, patent_date,patent_doc_kind, pat_number_of_days,length_of_grant,references_count,number_of_claims' $year_filename
    
    #copy to s3
    !aws s3 cp /home/ubuntu/data/output/ s3://ncw210data/PatentGrantData/ --recursive --only-show-errors  
        
    #!sleep 30
    

Loaded file:  /home/ubuntu/data/patentgrantloadzone/ipgb20160105.xml
Loaded file:  /home/ubuntu/data/patentgrantloadzone/ipgb20160112.xml
Loaded file:  /home/ubuntu/data/patentgrantloadzone/ipgb20160119.xml
Loaded file:  /home/ubuntu/data/patentgrantloadzone/ipgb20160126.xml
Loaded file:  /home/ubuntu/data/patentgrantloadzone/ipgb20160202.xml
Loaded file:  /home/ubuntu/data/patentgrantloadzone/ipgb20160209.xml
Loaded file:  /home/ubuntu/data/patentgrantloadzone/ipgb20160216.xml
Loaded file:  /home/ubuntu/data/patentgrantloadzone/ipgb20160223.xml
Loaded file:  /home/ubuntu/data/patentgrantloadzone/ipgb20160301.xml
Loaded file:  /home/ubuntu/data/patentgrantloadzone/ipgb20160308.xml
Loaded file:  /home/ubuntu/data/patentgrantloadzone/ipgb20160315.xml
Loaded file:  /home/ubuntu/data/patentgrantloadzone/ipgb20160322.xml
Loaded file:  /home/ubuntu/data/patentgrantloadzone/ipgb20160329.xml
Loaded file:  /home/ubuntu/data/patentgrantloadzone/ipgb20160405.xml
Loaded file:  /home/ubuntu/data/pa